In [2]:
import pandas as pd

In [3]:
df= pd.read_csv("./dataset/Reviews.csv")

In [4]:
df=df.drop(['Id','ProductId','UserId','ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Time','Summary'],axis='columns')

In [5]:
df.isna().sum()
df.shape

(568454, 2)

In [6]:
df.dropna(inplace=True)

In [7]:
df.duplicated().sum()

174779

In [8]:
df.drop_duplicates(inplace=True)

In [9]:
df['Score'].value_counts()/len(df)*100

Score
5    63.693402
4    14.243729
1     9.215724
3     7.562583
2     5.284562
Name: count, dtype: float64

In [10]:
df = df.groupby('Score', group_keys=False).apply(lambda x: x.sample(10000)).reset_index(drop=True)

C:\Users\anujn\AppData\Local\Temp\ipykernel_12352\3249606308.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby('Score', group_keys=False).apply(lambda x: x.sample(10000)).reset_index(drop=True)


In [11]:
df.reset_index(drop=True, inplace=True)
k = len(df)  # Ensure k matches the number of rows in df
df.index = range(1, k + 1)

In [12]:
import numpy as np
df['Score'].unique()

array([1, 2, 3, 4, 5], dtype=int64)

In [13]:
df['X'] = df['Text']

In [14]:
df=df.drop(['Text'],axis='columns')

In [15]:
score_mapping = {1: 0, 2: 0, 3: 1, 4: 2, 5: 2}

# Apply the mapping
df['Score'] = df['Score'].map(score_mapping)

In [16]:
X=df['X']

In [17]:
X=X.str.lower().replace(r'\w\s',' ').replace(r'\n',' ',regex=True)

In [18]:
import nltk 
from nltk.stem import PorterStemmer,WordNetLemmatizer

In [19]:
stemmer = WordNetLemmatizer()

In [20]:
def token(txt):
    token = nltk.word_tokenize(txt)
    a=[stemmer.lemmatize(w) for w in token]
    return ' '.join(a)

In [21]:
stemmer2 = PorterStemmer()

In [22]:
def token2(txt):
    token = nltk.word_tokenize(txt)
    a=[stemmer2.stem(w) for w in token]
    return ' '.join(a)

In [23]:
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer

In [24]:
X.apply(token)
X

1        1-10 count box is about $9 at vons...why is th...
2        i actually bought these at costco and was just...
3        the on-sale vita product peachy-mango was a ho...
4        i really love gummi candies of all kinds & fla...
5        compare to teavana white earl grey it is zero ...
                               ...                        
49996    my vet recommended evo. i ordered it even thou...
49997    i love this the fact that you can get a differ...
49998    my 1 year old siberian cat, mimic, loves this ...
49999    make sure you only steep it for only 0ne min. ...
50000    before detailing the added benefits of miso-cu...
Name: X, Length: 50000, dtype: object

In [25]:
X.apply(token2)

1        1-10 count box is about $ 9 at von ... whi is ...
2        i actual bought these at costco and wa just cu...
3        the on-sal vita product peachy-mango wa a hope...
4        i realli love gummi candi of all kind & flavor...
5        compar to teavana white earl grey it is zero v...
                               ...                        
49996    my vet recommend evo . i order it even though ...
49997    i love thi the fact that you can get a differ ...
49998    my 1 year old siberian cat , mimic , love thi ...
49999    make sure you onli steep it for onli 0ne min ....
50000    befor detail the ad benefit of miso-cup , i of...
Name: X, Length: 50000, dtype: object

In [26]:
cv=CountVectorizer(max_features=1000,stop_words='english')

In [27]:
matrix=cv.fit_transform(X).toarray()

In [28]:
p=cv.get_feature_names_out()
df2=pd.DataFrame(matrix,columns=p)

In [28]:
df2

,00,10,100,11,12,14,15,16,18,20,...,wrapped,wrong,www,year,years,yellow,yes,yogurt,yummy,zero
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,2,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
49997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
49998,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [29]:
y=df['Score']
X=df2

In [30]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
from sklearn.linear_model import LogisticRegression
#from sklearn.ensemble import RandomForest
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn.model_selection import train_test_split

In [31]:
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.2)

In [32]:
model =SVC(kernel='rbf',C=10)

In [33]:
model.fit(X_train,y_train)

SVC(C=10)

In [34]:
model.score(X_test,y_test)

0.6564

In [29]:
p.shape

(1000,)

In [30]:
print(p)

['00' '10' '100' '11' '12' '14' '15' '16' '18' '20' '24' '25' '30' '40'
 '50' '99' 'able' 'absolutely' 'acid' 'actual' 'actually' 'add' 'added'
 'adding' 'addition' 'advertised' 'aftertaste' 'ago' 'agree' 'air'
 'almond' 'almonds' 'alternative' 'amazing' 'amazon' 'american' 'anymore'
 'apart' 'apparently' 'apple' 'area' 'aren' 'aroma' 'arrived' 'artificial'
 'asked' 'ate' 'available' 'average' 'avoid' 'away' 'awesome' 'awful'
 'baby' 'bacon' 'bad' 'bag' 'bags' 'baked' 'baking' 'balance' 'banana'
 'bar' 'barely' 'bars' 'base' 'based' 'basically' 'basket' 'batch' 'bean'
 'beans' 'beef' 'believe' 'benefits' 'berry' 'best' 'better' 'beverage'
 'beware' 'big' 'bigger' 'bit' 'bite' 'bits' 'bitter' 'black' 'bland'
 'blend' 'blood' 'blue' 'blueberry' 'body' 'bold' 'bone' 'bones' 'boost'
 'bottle' 'bottles' 'bought' 'bowl' 'box' 'boxes' 'br' 'brand' 'brands'
 'bread' 'break' 'breakfast' 'brew' 'brewed' 'brewing' 'bring' 'broken'
 'broth' 'brown' 'bulk' 'butter' 'buy' 'buying' 'caffeine' 'cake' 